<a href="https://colab.research.google.com/github/caseydv/fashion_mnist/blob/main/densenet_rde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/MyDrive/Colab Notebooks")


Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D, Dense, concatenate, AveragePooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation, Dropout
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adadelta, Adam
from keras.optimizers import Adam
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from densenet import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
densenet = DenseNet((28,28,3), nb_classes=10, depth=35)

model = densenet.build_model()

model_optimizer = Adam(lr=1E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(loss='categorical_crossentropy', optimizer=model_optimizer)

earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')
mcp_save = ModelCheckpoint('denseb.h5', save_best_only=True, monitor='val_loss', verbose =1,mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
model.fit(x_train, y_train, batch_size=64, epochs=100, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.11)

model.save_weights('dense.h5')


In [22]:
train_original = pd.read_csv("image_train_Kaggle.csv")
blind_test_original = pd.read_csv("image_test_Kaggle.csv")
X, y = train_original.iloc[:, 1:], train_original.iloc[:,0]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=7980)
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y = to_categorical(y)

# Convert flatted data to the image format. 
X_train = X_train.values.reshape(X_train.shape[0], 28, 28)
X_val = X_val.values.reshape(X_val.shape[0], 28, 28)
blind_test = blind_test_original.drop(['ID'], axis=1).values.astype('float32')
blind_test = blind_test.reshape(blind_test.shape[0], 28, 28) / 255.0
X = X.values.reshape(X.shape[0], 28, 28)
X_train = np.repeat((X_train / X_train.max()).astype("float32")[..., np.newaxis], 3, -1)
X_val = np.repeat((X_val / X_val.max()).astype("float32")[..., np.newaxis], 3, -1)
blind_test = np.repeat((blind_test / blind_test.max()).astype("float32")[..., np.newaxis], 3, -1)
X = np.repeat((X / X.max()).astype("float32")[..., np.newaxis], 3, -1)
 

In [25]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

def random_reverse(x): 
    if np.random.random() > 0.5: 
        return x[:,::-1]
    else: 
        return x

def data_generator(X, Y, batch_size=100): 
    while True: 
        idxs = np.random.permutation(len(X))
        X = X[idxs]
        Y = Y[idxs]
        p, q = [], []
        for i in range(len(X)): 
            p.append(random_reverse(X[i]))
            q.append(Y[i])
            if len(p) == batch_size: 
                yield np.array(p), np.array(q)
                p, q = [], []
        if p: 
            yield np.array(p), np.array(q)
            p, q = [], []

datagen = ImageDataGenerator(
    featurewise_center=False,   
    samplewise_center=False,  
    featurewise_std_normalization=False,   
    samplewise_std_normalization=False,  
    zca_whitening=False,  
    rotation_range=0,  
    width_shift_range=0.1,   
    height_shift_range=0.1,  
    horizontal_flip=True, 
    vertical_flip=False,  
    preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False))

model.fit_generator(datagen.flow(X_train, y_train, batch_size=256), 
                                      steps_per_epoch=X_train.shape[0]//256, epochs=100, 
                                      validation_data=(X_val, y_val), callbacks=[mcp_save,reduce_lr_loss,earlyStopping])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
187/187 [==============================] - 54s 281ms/step - loss: 0.4859 - val_loss: 0.6196

Epoch 00001: val_loss improved from 0.86580 to 0.61957, saving model to denseb.h5
Epoch 2/100
187/187 [==============================] - 51s 273ms/step - loss: 0.4408 - val_loss: 0.5613

Epoch 00002: val_loss improved from 0.61957 to 0.56129, saving model to denseb.h5
Epoch 3/100
187/187 [==============================] - 51s 273ms/step - loss: 0.4081 - val_loss: 0.4475

Epoch 00003: val_loss improved from 0.56129 to 0.44750, saving model to denseb.h5
Epoch 4/100
187/187 [==============================] - 51s 273ms/step - loss: 0.3858 - val_loss: 0.4789

Epoch 00004: val_loss did not improve from 0.44750
Epoch 5/100
187/187 [==============================] - 51s 273ms/step - loss: 0.3709 - val_loss: 0.4101

Epoch 00005: val_loss improved from 0.44750 to 0.41006, saving model to denseb.h5
Epoch 6/100
187/187 [==============================] - 51s 273ms/step - loss: 0.3536 - val_loss:

In [26]:
model.load_weights("denseb.h5") 
blind_test_result = np.argmax(model.predict(blind_test), axis=-1)

my_submission = pd.DataFrame({'ID': blind_test_original.ID, 'Label': blind_test_result})
# you could use any filename. We choose submission here
my_submission.to_csv('output-rde.csv', index=False)